In [ ]:
'''
#Make Test catalog: Mars data

import os
import pandas as pd

main_path = 'data/mars/test/data/'

list_dfs = []

for root, dirs, files in os.walk(main_path):
    for file in files:
        if file.endswith('.csv'):
            trackfile = os.path.join(root, file)
            df = pd.read_csv(trackfile)
            df['filename'] = file
            list_dfs.append(df)

#Concat all Dataframes
df_concat = pd.concat(list_dfs, ignore_index=True)

#Save
df_concat.to_csv('data/mars/test/data/mars.csv', index=False)

print('Your new CSV with all Test data CSV is ready!')
'''

In [1]:
import pandas as pd
from obspy import read
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
from scipy import stats
import numpy as np

In [5]:
data_mars = pd.read_csv('mars.csv')
data_mars.head()

,time(%Y-%m-%dT%H:%M:%S.%f),rel_time(sec),velocity(c/s),filename
0,2019-05-23T02:00:00.032000,0.00,-0.000000,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
1,2019-05-23T02:00:00.082000,0.05,0.000199,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
2,2019-05-23T02:00:00.132000,0.10,-0.001630,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
3,2019-05-23T02:00:00.182000,0.15,-0.000875,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
4,2019-05-23T02:00:00.232000,0.20,-0.006137,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv


In [8]:
scaler = MinMaxScaler()
data_mars['velocity(c/s)'] = scaler.fit_transform(data_mars[['velocity(c/s)']])
window_size = 10  # Definir o tamanho da janela de suavização
data_mars['velocity_smooth'] = data_mars['velocity(c/s)'].rolling(window=window_size).mean()

peaks, _ = find_peaks(data_mars['velocity_smooth'], height=0.01)  
data_mars['is_sismo'] = 0
data_mars.loc[peaks, 'is_sismo'] = 1  

data_mars['log_velocity'] = np.log1p(data_mars['velocity(c/s)'].abs()) 

data_mars['z_score'] = np.abs(stats.zscore(data_mars['velocity(c/s)']))
sismos_mars = data_mars[data_mars['z_score'] > 0.1]  #Considering z-score = 0.1
sismos_mars

,time(%Y-%m-%dT%H:%M:%S.%f),rel_time(sec),velocity(c/s),filename,velocity_smooth,is_sismo,log_velocity,z_score
486533,2021-12-24T22:45:26.691000,2726.65,0.551322,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv,0.550205,0,0.439108,0.105423
486534,2021-12-24T22:45:26.741000,2726.70,0.551303,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv,0.550295,1,0.439095,0.103723
486549,2021-12-24T22:45:27.491000,2727.45,0.551293,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv,0.550328,1,0.439089,0.102916
486599,2021-12-24T22:45:29.991000,2729.95,0.551480,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv,0.550219,0,0.439209,0.119240
486600,2021-12-24T22:45:30.041000,2730.00,0.551652,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv,0.550346,0,0.439320,0.134312
...,...,...,...,...,...,...,...,...
647398,2022-05-04T23:59:29.948000,3569.90,0.548620,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv,0.548602,0,0.437364,0.130915
647399,2022-05-04T23:59:29.998000,3569.95,0.548559,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv,0.548577,0,0.437325,0.136318
647449,2022-05-04T23:59:32.498000,3572.45,0.551278,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv,0.550882,0,0.439079,0.101521
647478,2022-05-04T23:59:33.948000,3573.90,0.551343,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv,0.550818,0,0.439121,0.107237


In [10]:
# Data manipulation
sismos_mars.loc[:, 'rel_time(sec)'] = data_mars['rel_time(sec)'].astype(float).round(0).astype(int)
sismos_mars.loc[:, 'velocity(c/s)'] = data_mars['velocity(c/s)'].astype(float).round(0).astype(int)

In [11]:
np.float = np
np.int = np

In [12]:
import numpy as np
import pandas as pd
from skmultiflow.data import DataStream
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.evaluation import EvaluatePrequential
from scipy.signal import find_peaks
from sklearn.preprocessing import MinMaxScaler
from scipy import stats


X = sismos_mars[['rel_time(sec)', 'velocity(c/s)']].values
y = sismos_mars['is_sismo'].values  

stream = DataStream(X, y)

model = AdaptiveRandomForestClassifier()

evaluator = EvaluatePrequential(
    pretrain_size=100,  
    max_time=1000, 
    max_samples=len(X),  
    batch_size=1,  
    n_wait=100,  
    show_plot=False,  
    metrics=['accuracy', 'precision', 'recall', 'f1']  
)

evaluator.evaluate(stream=stream, model=[model], model_names=['ARF'])

evaluation_summary = evaluator.get_mean_measurements()
print(evaluation_summary)

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...
 #################### [100%] [812.41s]
Processed samples: 28384
Mean performance:
ARF - Accuracy     : 0.7668
ARF - Precision: 0.0589
ARF - Recall: 0.0097
ARF - F1 score: 0.0167
